# Récapitulation extractive de textes avec Python
Cette méthode utilise **Scikit-learn** et **spaCy** pour sélectionner les phrases les plus importantes dans un texte donné. Les phrases vont être gardées en entier.  
Une phrase est délimitée par un **.** ou **;**.  
#### Avantage  
Permettrait un résumé plus robuste - puisque l'algorithme aura des informations extérieures au texte, les STOP_WORDS.
#### Désavantage
Le désavantage est qu'il y a plus d'installations à faire.  

Installation des librairies:
- `$ pip install -U spacy`
- `$ pip install -U scikit-learn`  

Télécharger les langages voulus:
- Anglais `$ python -m spacy download en_core_news_sm`
- Portugais `$ python -m spacy download pt_core_news_sm`
- Français `$ python -m spacy download fr_core_news_sm`

In [16]:
# Imports
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer

# Load the packages by name with Spacy built in load()
nlpPT = spacy.load("pt_core_news_sm")
nlpFR = spacy.load("fr_core_news_sm")
nlpEN = spacy.load("en_core_web_sm")

In [17]:
# Get content from txt examples
with open('frankEN.txt') as f: textEN = f.read()
docEN = nlpEN(textEN)
with open('frankFR.txt') as f: textFR = f.read()
docFR = nlpFR(textFR)
with open('frankPT.txt') as f: textPT = f.read()
docPT = nlpPT(textPT)

Cette étape n'est pas sur ce notebook, mais il faudrait aussi une étape **preprocess** ? 
Ou est-ce que c'est déjà fait par le widget en question ?

In [21]:
# We will continue with the portuguese example
corpus = [sent.text.lower() for sent in docPT.sents ]
# Remove stopwords: provide little to no useful information
cv = CountVectorizer(stop_words=list(STOP_WORDS))   

cv_fit=cv.fit_transform(corpus)  

# Count unique words and how many times they appear
word_list = cv.get_feature_names();    
count_list = cv_fit.toarray().sum(axis=0)

# Create dictionnary of word frequency
word_frequency = dict(zip(word_list,count_list))

## Sort the dictionnary
This next cell will first **sort the dictionnary** we juste created,  
then get the **relative frequency** of words. 

In [20]:
# Get sorted dict of word frequency and print the top to test
val=sorted(word_frequency.values())
higher_word_frequencies = [word for word,freq in word_frequency.items() if freq in val[-3:]]
print("\nWords with higher frequencies: ", higher_word_frequencies)

# gets relative frequency of words
higher_frequency = val[-1]
for word in word_frequency.keys():  
    word_frequency[word] = (word_frequency[word]/higher_frequency)


Words with higher frequencies:  ['amigo', 'anos', 'outro', 'tenha']


In [30]:
# Initialise a sentence dictionnary
sentence_rank={}

# For each word in each sentence ... 
for sent in docPT.sents:
    for word in sent :    
        # if the word appears in word_frequency dict
        if word.text.lower() in word_frequency.keys(): 
            # Add "points"
            if sent in sentence_rank.keys():
                sentence_rank[sent]+=word_frequency[word.text.lower()]
            else:
                sentence_rank[sent]=word_frequency[word.text.lower()]
                
# Sort sentences
top_sentences=(sorted(sentence_rank.values())[::-1])
# This is where we can choose how many sentences we want to keep for the summary
top_sent=top_sentences[:3]

[30, 24, 23, 22, 22]

In [31]:
# We can now create a summary from those sentences:
summary=[]
for sent,strength in sentence_rank.items():  
    if strength in top_sent:
        summary.append(sent)
    else:
        continue
for i in summary:
    print(i,end=" ")

Não tenho ninguém próximo a mim, sereno e corajoso, que tenha uma mentalidade elevada e aberta, cujas aptidões sejam iguais às minhas, para aprovar ou corrigir meus planos. e eu realmente anseio por um amigo que tenha discernimento suficiente para não me ver como um sonhador e paciência para ajudar-me a organizar minhas idéias. Passei a juventude em solidão, vivi meus melhores anos em sua suave e feminina companhia, e isso moldou meu caráter de tal forma que sou incapaz de superar o desgosto intenso que me causa a brutalidade, tão comum nos navios. Há alguns anos ele amou uma jovem senhora russa de pequena fortuna e, como ele havia ganho uma considerável quantia em dinheiro, o pai da moça consentiu no casamento. Na prática, sou muito ativo, trabalhador, um operário pronto a executar tudo com perseverança, mas ao lado disso há um amor, uma crença no assombroso inserida em todos os meus projetos, que me coloca distante dos caminhos normais dos homens, impelindo-me para o mar bravo. 

This method will also work with other langages, as long as the correct nlp and correct STOP_WORDS list is used. 